# CatBoost with Aggressive Feature Selection (Top 10)

**Problem**: The CV-LB gap is ~10x. The intercept (0.0533) is 3x higher than target (0.01727).

**Hypothesis**: Simpler models with fewer features generalize better. Analysis showed that reducing features from 18 to 10 improved CV MSE by 0.84% with CatBoost.

**Approach**:
- Use CatBoostRegressor with `loss_function='RMSE'`
- Use ONLY the Top 10 features identified: `['Time', 'Temp', '1000/T', '1000/T*ln(t)', 'Spange_2', 'Spange_3', 'Spange_5', 'Spange_8', 'Spange_9', 'Spange_12']`
- Train separate models for each target (Product 2, Product 3, SM)
- Use robust CV scheme (LOO/LORO)

**Baseline**: exp_026 CV 0.008465, LB 0.0887

In [1]:
# Standard imports
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [2]:
# Load feature lookups - ONLY Spange needed for Top 10 features
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)

print(f'Spange: {SPANGE_DF.shape}')
print('Using Top 10 features only')

Spange: (26, 13)
Using Top 10 features only


In [3]:
# Feature Engineering for Top 10 Features
class Top10Featurizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        # Top 10 features: Time, Temp, 1000/T, 1000/T*ln(t), Spange_2, 3, 5, 8, 9, 12
        # Spange indices (0-based): 1, 2, 4, 7, 8, 11 (corresponding to 2, 3, 5, 8, 9, 12)
        self.spange_indices = [1, 2, 4, 7, 8, 11] 
        self.feats_dim = 4 + len(self.spange_indices) # 10 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        time_m = X_vals[:, 0:1]
        temp_c = X_vals[:, 1:2]
        
        # Kinetic features
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        
        # [Time, Temp, 1000/T, 1000/T*ln(t)]
        X_kinetic = np.hstack([time_m, temp_c, inv_temp, interaction])
        
        # Select specific Spange columns using iloc[:, indices]
        spange_subset = self.spange_df.iloc[:, self.spange_indices]
        
        if self.mixed:
            A_spange = spange_subset.loc[X["SOLVENT A NAME"]].values
            B_spange = spange_subset.loc[X["SOLVENT B NAME"]].values
            
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            # Select specific Spange columns
            X_spange = spange_subset.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Top10Featurizer defined with {Top10Featurizer().feats_dim} features')

Top10Featurizer defined with 10 features


In [4]:
# CatBoost Wrapper
class CatBoostWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = Top10Featurizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.models = []
        # Train separate model for each target
        for i in range(3):
            model = CatBoostRegressor(
                iterations=500, 
                learning_rate=0.05, 
                depth=6, 
                loss_function='RMSE',
                random_seed=42, 
                verbose=0,
                allow_writing_files=False
            )
            model.fit(X_all, y_all[:, i])
            self.models.append(model)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize(X_test, flip=False)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
        
        preds = []
        for i, model in enumerate(self.models):
            pred = model.predict(X_feat)
            
            if self.data_type == 'full':
                pred_flip = model.predict(X_flip)
                pred = (pred + pred_flip) / 2
                
            preds.append(pred)
        
        return np.column_stack(preds)

print('CatBoostWrapper defined')

CatBoostWrapper defined


In [5]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = CatBoostWrapper(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:00,  1.80it/s]

2it [00:01,  1.94it/s]

3it [00:01,  1.95it/s]

4it [00:02,  1.99it/s]

...

21it [00:10,  2.09it/s]

22it [00:10,  2.09it/s]

23it [00:11,  2.09it/s]

24it [00:11,  2.08it/s]

24it [00:11,  2.08it/s]

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = CatBoostWrapper(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nexp_026 baseline (MLP+LGBM): CV 0.008465, LB 0.0887')
print(f'This (CatBoost Top 10): CV {overall_mse:.6f}')

if overall_mse < 0.008465:
    improvement = (0.008465 - overall_mse) / 0.008465 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_026!')
else:
    degradation = (overall_mse - 0.008465) / 0.008465 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_026')